In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split

In [64]:
data=pd.read_parquet('/home/hashmi/Files/UpgradeSeedClassifier/InitialAnalysis/data/true_no_hits_15114101.parquet')[
    ['track_chi2PerDoF','track_nLHCbIDs','track_p',
     'track_phi','track_position_x', 'track_position_y',
     'track_position_r','track_tx', 'track_ty',
     'track_pseudoRapidity','particle_isDown_noVelo']].sample(frac=0.01,random_state=14)

In [65]:
data.head()

,track_chi2PerDoF,track_nLHCbIDs,track_p,track_phi,track_position_x,track_position_y,track_position_r,track_tx,track_ty,track_pseudoRapidity,particle_isDown_noVelo
100,1.450902,12.0,5968.964872,0.034813,299.038727,53.697536,303.821634,0.169439,0.005901,2.474912,False
103,1.126520,12.0,2470.408247,0.051994,1496.910400,254.066284,1518.318288,0.576064,0.029979,1.317719,False
39,1.588973,11.0,1996.761659,3.110768,-1332.257202,194.717667,1346.411610,-0.654579,0.020184,1.209633,False
228,1.148735,11.0,3893.033204,-3.099793,-1315.430908,-128.437622,1321.686308,-0.396475,-0.016582,1.654642,True
100,0.071442,12.0,5043.821056,-0.018976,562.949829,-28.173853,563.654394,0.232170,-0.004406,2.166470,False


In [66]:
x=data.drop('particle_isDown_noVelo',axis=1)

In [67]:
y=data['particle_isDown_noVelo']

In [68]:
x_train,x_test,y_train,y_test=train_test_split(x,y,stratify=y,random_state=14)

In [69]:
class DataFun(Dataset):
    def __init__(self,x_data,y_data):
        self.x_data=x_data
        self.y_data=y_data
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return len(self.x_data)

In [70]:
train_data=DataFun(torch.FloatTensor(x_train.values),torch.FloatTensor(y_train.values))

In [71]:
train_loader=DataLoader(train_data,batch_size=4,shuffle=True)
#test_loader=DataLoader(x_train,batch_size=1,shuffle=True)

In [72]:
## According to the paper:
class LogModel(nn.Module):
    def __init__(self,input_num,output_num):
        super(LogModel,self).__init__()
        self.Layer1=nn.Linear(input_num,15)
        self.relu=nn.ReLU()
        self.Layer2=nn.Linear(15,output_num)
        self.Sigmoid=nn.Sigmoid()
    def forward(self,x):
        output=self.Layer1(x)
        output=self.relu(output)
        output=self.Layer2(output)
#         output=self.Sigmoid(output)
        return output

In [73]:
model=LogModel(10,1)

In [74]:
epochs=10
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(params=model.parameters())

In [75]:
model.train()

LogModel(
  (Layer1): Linear(in_features=10, out_features=15, bias=True)
  (relu): ReLU()
  (Layer2): Linear(in_features=15, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
)

In [76]:
for epoch in range(1,epochs+1):
    epoch_loss=0
        
    for x_batch,y_batch in train_loader:
        optimizer.zero_grad()
        y_pred=model(x_batch)
        loss=criterion(y_batch.unsqueeze(1),y_pred)
        loss.backward()
        optimizer.step()
        epoch_loss+=loss.item()
    print(f'Epoch is {epoch} and Loss is {epoch_loss}')
        
        

Epoch is 1 and Loss is -8607094682.27852
Epoch is 2 and Loss is -51618170133.64948
Epoch is 3 and Loss is -132034531829.47063
Epoch is 4 and Loss is -249177615174.39334
Epoch is 5 and Loss is -403414742790.0062
Epoch is 6 and Loss is -593740262257.9838
Epoch is 7 and Loss is -818532536239.9946
Epoch is 8 and Loss is -1081477348662.489
Epoch is 9 and Loss is -1381498286622.3525
Epoch is 10 and Loss is -1713576861155.8179
